## Interaction

In [ ]:
%pip install ipyleaflet geopandas rasterio shapely folium ipywidgets

In [ ]:
# Paths 
site_path = "sample_data/"
img_path = site_path + "random_oilpalm.tif"
crowns_path = site_path + "predicted_crowns_map15.gpkg"

In [ ]:
# --- Imports ---
import rasterio
import geopandas as gpd
import numpy as np
from shapely.geometry import box, mapping, shape
from shapely.ops import transform
import pyproj
from PIL import Image

from ipyleaflet import (
    Map, ImageOverlay, GeoJSON, LayersControl, TileLayer, basemaps, basemap_to_tiles

)
from ipywidgets import HTML

# --- Paths ---
site_path = "sample_data/"
img_path = site_path + "random_oilpalm.tif"
crowns_path = site_path + "predicted_crowns_map15.gpkg"
overlay_img_path = site_path + "overlay_rgb.png"

# --- Save RGB overlay as PNG from GeoTIFF ---
def save_rgb_overlay_as_png(img_path, output_img="overlay.png"):
    with rasterio.open(img_path) as src:
        img = src.read()
        bounds = src.bounds
        crs = src.crs

        if img.shape[0] >= 3:
            rgb = np.stack([img[0], img[1], img[2]], axis=-1)
        else:
            raise ValueError("Need at least 3 bands for RGB")

        # Normalize for better display
        rgb = rgb.astype(np.float32)
        rgb_min, rgb_max = np.percentile(rgb[rgb > 0], (2, 98))
        rgb = np.clip((rgb - rgb_min) / (rgb_max - rgb_min), 0, 1)
        rgb = (rgb * 255).astype(np.uint8)

        image = Image.fromarray(rgb)
        image.save(output_img)

        return output_img, bounds, crs

overlay_img_path, utm_bounds, utm_crs = save_rgb_overlay_as_png(img_path)

# --- Convert UTM bounds to WGS84 ---
def reproject_bounds_to_wgs84(bounds, src_crs):
    project = pyproj.Transformer.from_crs(src_crs, "EPSG:4326", always_xy=True).transform
    return transform(project, box(*bounds)).bounds

wgs84_bounds = reproject_bounds_to_wgs84(utm_bounds, utm_crs)
lon_min, lat_min, lon_max, lat_max = wgs84_bounds

# --- Load crowns and reproject to WGS84 ---
crowns = gpd.read_file(crowns_path)
if crowns.crs != "EPSG:4326":
    crowns = crowns.to_crs("EPSG:4326")

# Assign feature IDs for deletion
crowns["id"] = crowns.index
crowns_json = crowns.__geo_interface__
for i, feat in enumerate(crowns_json["features"]):
    feat["id"] = i

# --- Initialize Map ---
center_lat = (lat_min + lat_max) / 2
center_lon = (lon_min + lon_max) / 2
m = Map(center=(center_lat, center_lon), zoom=18, max_zoom=20)
m.layout.width = '50%'  # Set width to 50% of the parent container
m.layout.height = '300px'  # Set height to 300 pixels
m.add_control(LayersControl(position="topright"))

# --- Google Satellite Base Layer ---
google_sat = TileLayer(
    url="http://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}",
    attribution="Google Satellite"
)
m.add_layer(google_sat)

# --- Esri Base Layer ---
esri_sat = basemap_to_tiles(basemaps.Esri.WorldImagery) # WorldImageryClarity, WorldImagery
m.add_layer(esri_sat)

# --- Image Overlay Layer ---
overlay = ImageOverlay(
    url=overlay_img_path,
    bounds=[(lat_min, lon_min), (lat_max, lon_max)],
    opacity=0.6,
    name="Oil Palm RGB"
)
m.add_layer(overlay)

# --- Crown Polygon GeoJSON Layer ---
popup = HTML()
crown_layer = GeoJSON(data=crowns_json, name="Crowns", hover_style={"fillColor": "red"})
m.popup = popup
deleted_ids = set()

def on_click_handler(event, feature, **kwargs):
    fid = feature.get("id")
    if fid is not None and fid not in deleted_ids:
        deleted_ids.add(fid)
        remaining = [f for f in crown_layer.data["features"] if f["id"] != fid]
        crown_layer.data["features"] = remaining
        popup.value = f"Deleted crown ID: {fid}"
    elif fid in deleted_ids:
        popup.value = f"Crown ID {fid} already deleted."

crown_layer.on_click(on_click_handler)
m.add_layer(crown_layer)

# --- Show Map ---
m

Map(center=[1.9545018283299727, 103.21470077403063], controls=(ZoomControl(options=['position', 'zoom_in_text'…